In [1]:
!test -d jumanpp-2.0.0-rc3 || curl -L https://github.com/ku-nlp/jumanpp/releases/download/v2.0.0-rc3/jumanpp-2.0.0-rc3.tar.xz | tar xJf -
!test -x /usr/local/bin/jumanpp || ( mkdir jumanpp-2.0.0-rc3/build && cd jumanpp-2.0.0-rc3/build && cmake .. -DCMAKE_BUILD_TYPE=Release && make install )

In [2]:
!jumanpp --version

Juman++ Version: 2.0.0-rc3 / Dictionary: 20190731-356e143 / LM: K:20190430-7d143fb L:20181122-b409be68 F:20171214-9d125cb


In [3]:
!pip install -U pip
!pip install pandas
!pip install transformers
!pip install torch torchvision torchaudio
!pip install datasets
!pip install pyknp
!pip install pytextspan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Tokenizerの実行結果確認

In [4]:
from transformers import AutoModelForQuestionAnswering
from transformers import AutoTokenizer
from transformers import BertJapaneseTokenizer
from transformers import DebertaV2TokenizerFast
from transformers.utils import cached_file
from tokenizers.pre_tokenizers import PreTokenizer, Metaspace, Sequence
from transformers.models.bert_japanese.tokenization_bert_japanese import JumanppTokenizer
from pyknp import Juman
import textspan

jumanpp = Juman()
model_id = 'ku-nlp/deberta-v2-base-japanese'
model = AutoModelForQuestionAnswering.from_pretrained(model_id)
text = '隣の客はよく柿食う客だ'

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(1, tokenizer.tokenize(text))

# tokenizer.word_tokenizer_type = 'jumanpp'
# print(2, tokenizer.tokenize(text))

# tokenizer = DebertaV2TokenizerFast(vocab_file=tokenizer.vocab_file, spm_file=cached_file(
#     model_id, 'spm.model'), word_tokenizer_type='jumanpp', subword_tokenizer_type='sentencepiece')
# tokenizer._tokenize = lambda text: [sub_token.replace('〓', tokenizer.mask_token) for token in tokenizer.word_tokenizer.tokenize(text.replace(
#     tokenizer.mask_token, '\u2581'), never_split=tokenizer.all_special_tokens) for sub_token in tokenizer.subword_tokenizer.tokenize(('\u2581'+token).replace('\u2581\u2581', '〓'))]

# print(3, tokenizer_juman.tokenize(text))

# tokenizer = BertJapaneseTokenizer(vocab_file=tokenizer.vocab_file, spm_file=cached_file(
#     model_id, 'spm.model'), word_tokenizer_type='jumanpp', subword_tokenizer_type='sentencepiece')
# tokenizer._tokenize = lambda text: [sub_token.replace("〓", tokenizer.mask_token) for token in tokenizer.word_tokenizer.tokenize(text.replace(
#     tokenizer.mask_token, "\u2581"), never_split=tokenizer.all_special_tokens) for sub_token in tokenizer.subword_tokenizer.tokenize(("\u2581"+token).replace("\u2581\u2581", "〓"))]

# print(4, tokenizer.tokenize(text))

tokenizer = AutoTokenizer.from_pretrained(model_id)
print(5, tokenizer.tokenize(' '.join([mrph.midasi for mrph in jumanpp.analysis(text).mrph_list()])))

tokenizer = DebertaV2TokenizerFast.from_pretrained(model_id)
class JumanppPreTokenizer(JumanppTokenizer):
    def jumanpp_split(self, i, normalized_string):
        t = str(normalized_string)
        k = self.tokenize(t)

        return [normalized_string[s:e] for c in textspan.get_original_spans(k,t) for s,e in c]
    def pre_tokenize(self, pretok):
        pretok.split(self.jumanpp_split)

tokenizer._tokenizer.pre_tokenizer = Sequence([PreTokenizer.custom(JumanppPreTokenizer()), Metaspace()])
print(6, tokenizer.tokenize(text))

Some weights of the model checkpoint at ku-nlp/deberta-v2-base-japanese were not used when initializing DebertaV2ForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2ForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at ku-nlp/debert

1 ['▁隣', 'の', '客', 'は', 'よく', '柿', '食', 'う', '客', 'だ']
5 ['▁隣', '▁の', '▁客', '▁は', '▁よく', '▁柿', '▁食', 'う', '▁客', '▁だ']
6 ['▁隣', '▁の', '▁客', '▁は', '▁よく', '▁柿', '▁食', 'う', '▁客', '▁だ']


## Auto Tokenizer

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import pandas as pd
import json
import numpy as np

train_filename = '/content/drive/MyDrive/Colab Notebooks/train-qas/data/jsquad-v1.0/train-v1.0.json'
test_filename = '/content/drive/MyDrive/Colab Notebooks/train-qas/data/jsquad-v1.0/valid-v1.0.json'


# https://www.kaggle.com/code/sanjay11100/squad-stanford-q-a-json-to-pandas-dataframe
def squad_json_to_dataframe_train(input_file_path, record_path=['data', 'paragraphs', 'qas', 'answers']):
    file = json.loads(open(input_file_path).read())
    m = pd.io.json.json_normalize(file, record_path[:-1])
    r = pd.io.json.json_normalize(file, record_path[:-2])

    # combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    m['context'] = idx
    m['answers'] = m['answers'].apply(lambda x: x[0])

    return m[['question', 'context', 'answers']]


train_df = squad_json_to_dataframe_train(train_filename).sample(3000, random_state=42)
test_df = squad_json_to_dataframe_train(test_filename).sample(300, random_state=42)
# train_df = squad_json_to_dataframe_train(train_filename)
# test_df = squad_json_to_dataframe_train(test_filename)

display(train_df)
display(test_df)

<ipython-input-6-48733318d594>:12: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  m = pd.io.json.json_normalize(file, record_path[:-1])
<ipython-input-6-48733318d594>:13: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  r = pd.io.json.json_normalize(file, record_path[:-2])


,question,context,answers
43444,パンタグラフによるトロリ線の押し上げ量を平均化するには、何を追加する必要があるか？,架空電車線方式 [SEP] パンタグラフによるトロリ線の押し上げ量を平均化する目的で、吊架線...,"{'text': '補助用吊架線', 'answer_start': 54}"
17526,ベラルーシ語は現在何という文字で表記されるか。,ベラルーシ語 [SEP] ベラルーシ語は現在キリル文字で表記される。キリル文字表記法としては...,"{'text': 'キリル文字', 'answer_start': 22}"
45299,関係者に何名のノーベル賞受賞者がいるか,名古屋大学 [SEP] 関係者に7名のノーベル賞受賞者がいる。そのうち、名古屋大学を主たる研...,"{'text': '7名', 'answer_start': 16}"
44189,読者の投稿した漢詩が毎号掲載されている雑誌は何か？,漢詩 [SEP] 現在でも自作の漢詩集を著している陳舜臣等、漢詩創作の愛好家は存在しており、...,"{'text': '月刊誌大法輪', 'answer_start': 46}"
38823,タリン、マリエハムン、ストックホルムへ様々な事業者が運航している航路は？,ヘルシンキ [SEP] フェリー航路はタリン、マリエハムン、ストックホルムへ様々な事業者が運...,"{'text': 'フェリー航路', 'answer_start': 12}"
...,...,...,...
23847,溶質が強塩基ではなく弱塩基の場合、考慮する必要があるのは塩基解離平衡と何か。,水素イオン指数 [SEP] 水酸化カリウムなどの他のアルカリ金属の水酸化物の場合も、アルカリ...,"{'text': '加水分解', 'answer_start': 138}"
42669,基本となる測量方法は基準点測量、水準測量、地形測量及び何？,測量 [SEP] 基準点測量、水準測量、地形測量及び写真測量などの基本となる測量方法を活用し...,"{'text': '写真測量', 'answer_start': 26}"
14768,解雇予告がなされても、その予告期間が満了するまでの間は、使用者は何の義務がありますか。,解雇 [SEP] 解雇予告がなされても、その予告期間が満了するまでの間は、労働関係は有効に存...,"{'text': '賃金支払', 'answer_start': 74}"
30459,5月25日にトリニティー・ユナイテッド教会に招かれた神父は誰か,バラク・オバマ [SEP] 5月25日にはトリニティー・ユナイテッド教会に招かれた神父のマイ...,"{'text': 'マイケル・フレガー', 'answer_start': 44}"


,question,context,answers
2259,政治問題が直接又は間接に雇用条件に影響を及ぼすことを否定できるか否か。,労働基本権 [SEP] もっとも政治目的での団体行動といえども、政治問題が直接又は間接に雇用...,"{'text': 'できない', 'answer_start': 60}"
2004,オランダで需要量のほぼ全量が輸入されている燃料は？,オランダ [SEP] 石油は北海油田で産出されているが国内需要量には届かないため、輸入が行わ...,"{'text': '石炭', 'answer_start': 51}"
1923,昭和天皇がオランダ歴訪したのはいつ？,オランダ [SEP] その後も反日感情は残存し、1971年の昭和天皇オランダ歴訪の際には、在...,"{'text': '1971年', 'answer_start': 24}"
3198,炭化水素をエネルギー源として利用した副産物は？,炭化水素 [SEP] 炭化水素をエネルギー源として利用した副産物の二酸化炭素も、メタンガスも...,"{'text': '二酸化炭素', 'answer_start': 33}"
843,ラオスの農業の基盤は,ラオス [SEP] 少ない人口が満遍なく分散して暮らすラオスでは、大部分の人は稲作を基盤とす...,"{'text': '稲作', 'answer_start': 39}"
...,...,...,...
3682,ポルトガル語諸国共同体の創立年は。,ポルトガル [SEP] EUとブラジル以外ではアンゴラやモザンビークなどの旧植民地諸国と関係...,"{'text': '1996年', 'answer_start': 50}"
2628,せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆる座標系の中で何が...,応力 [SEP] せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆ...,"{'text': '垂直応力', 'answer_start': 23}"
2815,建築家は語源学的に何から派生しているか？,建築家 [SEP] 建築を実践することは、建物の設計や建物を取り巻く敷地内の空間で、人間の占...,"{'text': 'ラテンのarchitectus', 'answer_start': 84}"
1212,第二次世界大戦が日本の降伏によって終わりを告げたのは西暦何年のことですか？,日本共産党 [SEP] 第二次世界大戦が1945年8月15日に日本の降伏で終結した後、10月...,"{'text': '1945年', 'answer_start': 20}"


In [7]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import AutoTokenizer


model_ckpt = 'ku-nlp/deberta-v2-base-japanese'

model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
max_length = 512  # The maximum length of a feature (question and context)
doc_stride = (
    0  # The authorized overlap between two part of the context when splitting
)


# https://huggingface.co/docs/transformers/tasks/question_answering
def prepare_train_features(examples):
    #
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    examples['question'] = [q.strip() for q in examples['question']]
    examples['context'] = [c.strip() for c in examples['context']]
    inputs = tokenizer(
        text=examples['question'],
        text_pair=examples['context'],
        truncation='only_second',
        max_length=max_length,
        stride=doc_stride,
        return_offsets_mapping=True,
        padding='max_length',
    )

    offset_mapping = inputs.pop('offset_mapping')
    answers = examples['answers']
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer['answer_start']
        end_char = answer['answer_start'] + len(answer['text'])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions

    return inputs


Some weights of the model checkpoint at ku-nlp/deberta-v2-base-japanese were not used when initializing DebertaV2ForQuestionAnswering: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing DebertaV2ForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaV2ForQuestionAnswering were not initialized from the model checkpoint at ku-nlp/debert

In [8]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)


In [9]:
tokenized_train_dataset = train_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=3,
)
tokenized_test_dataset = test_dataset.map(
    prepare_train_features,
    batched=True,
    remove_columns=test_dataset.column_names,
    num_proc=3,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator()


In [11]:
import os

os.environ['WANDB_DISABLED'] = 'true'

training_args = TrainingArguments(
    seed=42,
    output_dir='./outputs',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125
  Number of trainable parameters = 111800066


Epoch,Training Loss,Validation Loss
1,No log,1.352833
2,2.051200,1.179814
3,0.914700,1.192168


***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./outputs/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./outputs/checkpoint-500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./outputs/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./outputs/checkpoint-1000/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=1.3986566162109375, metrics={'train_runtime': 1303.029, 'train_samples_per_second': 6.907, 'train_steps_per_second': 0.863, 'total_flos': 2400699340800000.0, 'train_loss': 1.3986566162109375, 'epoch': 3.0})

In [12]:
from transformers import pipeline

pipe = pipeline('question-answering', model=trainer.model,
                tokenizer=trainer.tokenizer, device=0)


In [13]:
def predict(row):
    ret_df = pd.DataFrame()
    ret = pipe(question=row['question'], context=row['context'],
               top_k=1, handle_impossible_answer=False,  align_to_words=False)

    return ret['answer'], ret['score']


pred_df = test_df[['question', 'context', 'answers']].copy()
pred_df['actual_answer'] = pred_df['answers'].apply(lambda x: x['text'])
pred_df[['pred_answer', 'prob']] = pred_df.apply(
    predict, axis=1, result_type='expand')
pred_df = pred_df.drop('answers', axis=1)
pred_df['correct'] = pred_df.apply(
    lambda row: row['pred_answer'] == row['actual_answer'], axis=1)


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [14]:
pd.set_option('display.max_colwidth', 300)

print(f'Accuracy: {len(pred_df[pred_df.correct]) / len(pred_df)}')
display(pred_df)


Accuracy: 0.6266666666666667


,question,context,actual_answer,pred_answer,prob,correct
2259,政治問題が直接又は間接に雇用条件に影響を及ぼすことを否定できるか否か。,労働基本権 [SEP] もっとも政治目的での団体行動といえども、政治問題が直接又は間接に雇用条件に影響を及ぼすことは否定できないことに鑑みると、それを全く保障されないものとしてしまうことには問題がないわけではない。上の例で言えば武器輸出の拡大によって勤務時間が延長されたり、武器を国外に輸出されることによって勤労者の精神衛生に悪影響を及ぼすということもあり得るであろうし、危険な戦闘地域での輸送業務への従事は勤務内容そのものである。このため直接的影響がある場合には政治目的での団体行動は憲法による保障を受けるとしたり、あらゆる政治目的での団体行動が憲法28条によって保障されているとする見解も...,できない,否定できない,0.177811,False
2004,オランダで需要量のほぼ全量が輸入されている燃料は？,オランダ [SEP] 石油は北海油田で産出されているが国内需要量には届かないため、輸入が行われている。石炭は需要量のほぼ全量が輸入されている。石油小売のガソリンスタンドは、ロイヤル・ダッチ・シェル、BP、TEXACO、ESSOなどが国内寡占状態である。,石炭,石炭,0.213218,True
1923,昭和天皇がオランダ歴訪したのはいつ？,オランダ [SEP] その後も反日感情は残存し、1971年の昭和天皇オランダ歴訪の際には、在位中の昭和天皇はオランダ人にとって戦争犯罪人と見なされていたため市民から卵や魔法瓶を投げつけられるなどした。またベアトリクス女王が1986年に日本訪問の計画をした際には、議会と世論の反発で中止した。,1971年,1971年,0.983878,True
3198,炭化水素をエネルギー源として利用した副産物は？,炭化水素 [SEP] 炭化水素をエネルギー源として利用した副産物の二酸化炭素も、メタンガスも地球温暖化に関与している。また、内燃機関が不完全燃焼することで発生する一酸化炭素は大気汚染物質であり、窒素酸化物とともに光化学スモッグの原因ともなる。粒子状物質には高分子の炭化水素が含まれる。,二酸化炭素,二酸化炭素,0.931181,True
843,ラオスの農業の基盤は,ラオス [SEP] 少ない人口が満遍なく分散して暮らすラオスでは、大部分の人は稲作を基盤とする農業を営んでいる。まず、自給米を確保して余剰分を販売し、現金収入とする。ラオス人の主食はもち米である。自給農業を基盤とした分散型社会である。,稲作,稲作,0.655409,True
...,...,...,...,...,...,...
3682,ポルトガル語諸国共同体の創立年は。,ポルトガル [SEP] EUとブラジル以外ではアンゴラやモザンビークなどの旧植民地諸国と関係が深く、1996年にはポルトガル語諸国共同体（CPLP）を加盟国と共同で設立した。ポルトガルは1990年代からCPLP加盟国のアンゴラやモザンビークなどのルゾフォニア諸国にポルトガル語教師の派遣を行っており、東ティモールの独立後にも同国にさまざまな援助（特にポルトガル語教師の派遣）を行っている。,1996年,1996年,0.913312,True
2628,せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆる座標系の中で何が最大、最小となる値を示している？,応力 [SEP] せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆる座標系の中で垂直応力が最大、最小となる値を示している。3つの主応力をσ1 ≥ σ2 ≥ σ3 の関係となるようにとったとき、最大の主応力σ1 を最大主応力 、最小となる主応力σ3 を最小主応力 、これら2つに直交する主応力σ2 を中間主応力 と呼び、ある座標系での応力状態formula_26が与えられているとき、主応力は以下の関係から求められる。,垂直応力,最小主応力,0.184178,False
2815,建築家は語源学的に何から派生しているか？,建築家 [SEP] 建築を実践することは、建物の設計や建物を取り巻く敷地内の空間で、人間の占有や使用を主目的としたサービスを提供することを意味する。語源学的に、建築家はラテンのarchitectusから派生している。更にそれは、ギリシャ語のチーフ建設者（arkhi-、チーフ + tekton、建設者）から派生している。建築家は建築の作者のタイトルであり、都市環境、建築物、および屋内外の環境を計画および設計する職である。,ラテンのarchitectus,architectus,0.645247,False
1212,第二次世界大戦が日本の降伏によって終わりを告げたのは西暦何年のことですか？,日本共産党 [SEP] 第二次世界大戦が1945年8月15日に日本の降伏で終結した後、10月4日の治安維持法撤廃と政治犯釈放を要求するGHQ指令により約220名の共産党員が出獄し、徳田球一、宮本顕治、袴田里見、黒木重徳、志賀義雄らは合法政党として日本共産党を再建（書記長・徳田球一）、機関紙「赤旗」を再刊、党本部は千駄ヶ谷の溶接学校跡地に置いた。なお戦前の共産党（第二次共産党）との断絶を重視する立場（加藤哲郎など）からは、これ以降の共産党を「戦後共産党」（第三次共産党）とも称する。戦争に反対した共産党員の出獄は国民に歓迎された。,1945年,1945年,0.556352,True


## Juman Tokenizer

In [19]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import DebertaV2TokenizerFast, AutoModelForMaskedLM, FillMaskPipeline
from transformers.utils import cached_file

model_ckpt = 'ku-nlp/deberta-v2-base-japanese'

model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)
tokenizer = DebertaV2TokenizerFast.from_pretrained(model_ckpt)
class JumanppPreTokenizer(JumanppTokenizer):
    def jumanpp_split(self, i, normalized_string):
        t = str(normalized_string)
        k = self.tokenize(t)

        return [normalized_string[s:e] for c in textspan.get_original_spans(k,t) for s,e in c]
    def pre_tokenize(self, pretok):
        pretok.split(self.jumanpp_split)
tokenizer._tokenizer.pre_tokenizer = Sequence([PreTokenizer.custom(JumanppPreTokenizer()), Metaspace()])

max_length = 512  # The maximum length of a feature (question and context)
doc_stride = (
    0  # The authorized overlap between two part of the context when splitting
)

# https://huggingface.co/docs/transformers/tasks/question_answering
def prepare_train_features_juman(examples):
    #
    # Tokenize our examples with truncation and padding, but keep the overflows using a
    # stride. This results in one example possible giving several features when a context is long,
    # each of those features having a context that overlaps a bit the context of the previous
    # feature.
    examples['question'] = [q.strip() for q in examples['question']]
    examples['context'] = [c.strip() for c in examples['context']]

    inputs = tokenizer(
        text=examples['question'],
        text_pair=examples['context'],
        truncation='only_second',
        max_length=max_length,
        stride=doc_stride,
        return_offsets_mapping=True,
        padding='max_length',
    )

    offset_mapping = inputs.pop('offset_mapping')
    answers = examples['answers']
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer['answer_start']
        end_char = answer['answer_start'] + len(answer['text'])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs['start_positions'] = start_positions
    inputs['end_positions'] = end_positions

    return inputs


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--ku-nlp--deberta-v2-base-japanese/snapshots/126d42650caaff995e08d10ea62b14fc06afbfb4/config.json
Model config DebertaV2Config {
  "_name_or_path": "ku-nlp/deberta-v2-base-japanese",
  "architectures": [
    "DebertaV2ForMaskedLM"
  ],
  "attention_head_size": 64,
  "attention_probs_dropout_prob": 0.1,
  "conv_act": "gelu",
  "conv_kernel_size": 3,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "re

In [20]:
tokenized_train_dataset = train_dataset.map(
    prepare_train_features_juman,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=3,
)
tokenized_test_dataset = test_dataset.map(
    prepare_train_features_juman,
    batched=True,
    remove_columns=test_dataset.column_names,
    num_proc=3,
)

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
import os

os.environ['WANDB_DISABLED'] = 'true'

training_args = TrainingArguments(
    seed=42,
    output_dir='./outputs',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    save_strategy='no', # Custom PreTokenizerはそのままでは保存できないので、ここでは簡易のために保存しないようにします。
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
***** Running training *****
  Num examples = 3000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1125
  Number of trainable parameters = 111800066


Epoch,Training Loss,Validation Loss
1,No log,1.010052
2,1.563400,0.973378
3,0.573300,1.069897


***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8
***** Running Evaluation *****
  Num examples = 300
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1125, training_loss=1.003351318359375, metrics={'train_runtime': 1287.2362, 'train_samples_per_second': 6.992, 'train_steps_per_second': 0.874, 'total_flos': 2400699340800000.0, 'train_loss': 1.003351318359375, 'epoch': 3.0})

In [22]:
from transformers import pipeline

pipe = pipeline('question-answering', model=trainer.model,
                tokenizer=trainer.tokenizer, device=0)


In [23]:
def predict(row):
    ret_df = pd.DataFrame()
    ret = pipe(question=row['question'], context=row['context'],
               top_k=1, handle_impossible_answer=False,  align_to_words=False)

    return ret['answer'], ret['score']


pred_df = test_df[['question', 'context', 'answers']].copy()
pred_df['actual_answer'] = pred_df['answers'].apply(lambda x: x['text'])
pred_df[['pred_answer', 'prob']] = pred_df.apply(
    predict, axis=1, result_type='expand')
pred_df = pred_df.drop('answers', axis=1)
pred_df['correct'] = pred_df.apply(
    lambda row: row['pred_answer'] == row['actual_answer'], axis=1)


/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [24]:
pd.set_option('display.max_colwidth', 300)

print(f'Accuracy: {len(pred_df[pred_df.correct]) / len(pred_df)}')
display(pred_df)


Accuracy: 0.7033333333333334


,question,context,actual_answer,pred_answer,prob,correct
2259,政治問題が直接又は間接に雇用条件に影響を及ぼすことを否定できるか否か。,労働基本権 [SEP] もっとも政治目的での団体行動といえども、政治問題が直接又は間接に雇用条件に影響を及ぼすことは否定できないことに鑑みると、それを全く保障されないものとしてしまうことには問題がないわけではない。上の例で言えば武器輸出の拡大によって勤務時間が延長されたり、武器を国外に輸出されることによって勤労者の精神衛生に悪影響を及ぼすということもあり得るであろうし、危険な戦闘地域での輸送業務への従事は勤務内容そのものである。このため直接的影響がある場合には政治目的での団体行動は憲法による保障を受けるとしたり、あらゆる政治目的での団体行動が憲法28条によって保障されているとする見解も...,できない,労働基本権,0.228355,False
2004,オランダで需要量のほぼ全量が輸入されている燃料は？,オランダ [SEP] 石油は北海油田で産出されているが国内需要量には届かないため、輸入が行われている。石炭は需要量のほぼ全量が輸入されている。石油小売のガソリンスタンドは、ロイヤル・ダッチ・シェル、BP、TEXACO、ESSOなどが国内寡占状態である。,石炭,石炭,0.977070,True
1923,昭和天皇がオランダ歴訪したのはいつ？,オランダ [SEP] その後も反日感情は残存し、1971年の昭和天皇オランダ歴訪の際には、在位中の昭和天皇はオランダ人にとって戦争犯罪人と見なされていたため市民から卵や魔法瓶を投げつけられるなどした。またベアトリクス女王が1986年に日本訪問の計画をした際には、議会と世論の反発で中止した。,1971年,1971年,0.991722,True
3198,炭化水素をエネルギー源として利用した副産物は？,炭化水素 [SEP] 炭化水素をエネルギー源として利用した副産物の二酸化炭素も、メタンガスも地球温暖化に関与している。また、内燃機関が不完全燃焼することで発生する一酸化炭素は大気汚染物質であり、窒素酸化物とともに光化学スモッグの原因ともなる。粒子状物質には高分子の炭化水素が含まれる。,二酸化炭素,二酸化炭素,0.957254,True
843,ラオスの農業の基盤は,ラオス [SEP] 少ない人口が満遍なく分散して暮らすラオスでは、大部分の人は稲作を基盤とする農業を営んでいる。まず、自給米を確保して余剰分を販売し、現金収入とする。ラオス人の主食はもち米である。自給農業を基盤とした分散型社会である。,稲作,稲作,0.940181,True
...,...,...,...,...,...,...
3682,ポルトガル語諸国共同体の創立年は。,ポルトガル [SEP] EUとブラジル以外ではアンゴラやモザンビークなどの旧植民地諸国と関係が深く、1996年にはポルトガル語諸国共同体（CPLP）を加盟国と共同で設立した。ポルトガルは1990年代からCPLP加盟国のアンゴラやモザンビークなどのルゾフォニア諸国にポルトガル語教師の派遣を行っており、東ティモールの独立後にも同国にさまざまな援助（特にポルトガル語教師の派遣）を行っている。,1996年,1996年,0.846129,True
2628,せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆる座標系の中で何が最大、最小となる値を示している？,応力 [SEP] せん断応力がゼロとなるときの垂直応力が主応力であるが、同時に主応力はあらゆる座標系の中で垂直応力が最大、最小となる値を示している。3つの主応力をσ1 ≥ σ2 ≥ σ3 の関係となるようにとったとき、最大の主応力σ1 を最大主応力 、最小となる主応力σ3 を最小主応力 、これら2つに直交する主応力σ2 を中間主応力 と呼び、ある座標系での応力状態formula_26が与えられているとき、主応力は以下の関係から求められる。,垂直応力,応力,0.953400,False
2815,建築家は語源学的に何から派生しているか？,建築家 [SEP] 建築を実践することは、建物の設計や建物を取り巻く敷地内の空間で、人間の占有や使用を主目的としたサービスを提供することを意味する。語源学的に、建築家はラテンのarchitectusから派生している。更にそれは、ギリシャ語のチーフ建設者（arkhi-、チーフ + tekton、建設者）から派生している。建築家は建築の作者のタイトルであり、都市環境、建築物、および屋内外の環境を計画および設計する職である。,ラテンのarchitectus,ラテンのarchitectus,0.736200,True
1212,第二次世界大戦が日本の降伏によって終わりを告げたのは西暦何年のことですか？,日本共産党 [SEP] 第二次世界大戦が1945年8月15日に日本の降伏で終結した後、10月4日の治安維持法撤廃と政治犯釈放を要求するGHQ指令により約220名の共産党員が出獄し、徳田球一、宮本顕治、袴田里見、黒木重徳、志賀義雄らは合法政党として日本共産党を再建（書記長・徳田球一）、機関紙「赤旗」を再刊、党本部は千駄ヶ谷の溶接学校跡地に置いた。なお戦前の共産党（第二次共産党）との断絶を重視する立場（加藤哲郎など）からは、これ以降の共産党を「戦後共産党」（第三次共産党）とも称する。戦争に反対した共産党員の出獄は国民に歓迎された。,1945年,1945年,0.585925,True


## 参考文献

* [yasuokaの日記: ku-nlp/deberta-v2-base-japaneseのトークナイザをDebertaV2TokenizerFastのままでJuman++に繋ぐには](https://srad.jp/~yasuoka/journal/659674/)
* [日本語でHugging Face Tokenizersを動かす](https://tech.mntsq.co.jp/entry/2021/02/26/120013)
* [Huggingface で公開されている日本語モデルを使って QA タスクをファインチューニングする](https://qiita.com/hideki/items/394f120d8eea030cb552)